# step 1: Predefined CLass

In [1]:
import random

class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()

    def get_propaganda(self):
        if int(self.SUBJprop) <=3:
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE

class ReviewContainer:
    def __init__(self,rewievs):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [x.propaganda for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)





In [ ]:
nltk.download()

# step 2: Load Data

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from string import digits

reviews = []
data  = pd.read_excel('Data/Dataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))

print(reviews[0].sentence)
print(reviews[0].SUBJprop)
sentenceArray = [ x.sentence for x  in reviews] 
# print(sentenceArray[2])

# remove number
s= sentenceArray[0]
remove_digits = str.maketrans('', '', digits)
    # translate(remove_digits)
sentenceArray_num = [x.translate(remove_digits) for x in sentenceArray]
#print(sentenceArray_num[3])

# remove punctuation
translator = str.maketrans('', '', string.punctuation)
# s = 'string with "punctuation" inside of it! Does this work? I hope so.'
# print(s.translate(translator))
sentenceArray_pun = [x.translate(translator) for x in sentenceArray]
#print(sentenceArray_pun[0])

# remove white space
input_str = " \t a string example\t "
input_str = input_str.strip()
#print(input_str)
sentenceArray_white = [x.strip() for x in sentenceArray]
#print(sentenceArray_white[4])


miley and liam fighting false rumors swirl that theyre in a feud over a supposed prenup
1


# step 3: Prep Data (split into train and test set)

In [3]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, train_size=0.7, random_state = 42 ) 
# len(training)
# print(len(test)) # 70% of traning set

developmentSet, testSet = train_test_split(test, train_size=0.5, random_state = 42 ) 
# print(len(testSet)) # 15% of test set
# print(len(developmentSet)) # 15% of development set
# print(developmentSet[0].sentence)

train_container = ReviewContainer(training)
developmentSet_container = ReviewContainer(developmentSet)
testSet_container = ReviewContainer(testSet)

# step 4: Seperate the attribute, originally our array has text and score. we want them to be a seperate array


In [4]:
train_container.evenly_distribute()
train_sentence = train_container.get_sentence()    # return just text
train_propaganda = train_container.get_propaganda()   # return just sentiment

development_sentence = developmentSet_container.get_sentence() 
development_propaganda = developmentSet_container.get_propaganda() 

testSet_container.evenly_distribute()
test_sentence = testSet_container.get_sentence() 
test_propaganda = testSet_container.get_propaganda() 

print("Positive Propaganda:")
print(train_propaganda.count(Propaganda.POSITIVE))

print("\nNegative Propaganda:")
print(train_propaganda.count(Propaganda.NEGATIVE))


Positive Propaganda:
105

Negative Propaganda:
105


## step 4: Bag Of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_sentence) # return a matrix of either 0 or 1 # train_x is our text
test_x_vectors = vectorizer.transform(test_sentence)

# Todo:
# transform development set to vectors
development_x_vectors = vectorizer.fit_transform(development_sentence)
print(train_sentence[0])
print(train_x_vectors[0].toarray())
print(test_x_vectors[0].toarray())
print(development_x_vectors[0].toarray())


# step 5: Classification SVM

In [ ]:
from sklearn import svm

clf_svm = svm.SVC(kernel='rbf',C=1)
clf_svm.fit(train_x_vectors, train_propaganda)



clf_svm.predict(test_x_vectors[11])

# step 5: Decision Tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_propaganda)

clf_dec.predict(test_x_vectors[1])

# step 5: Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray() , train_propaganda)

clf_gnb.predict(test_x_vectors[1].toarray())

# step 5: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_propaganda)

clf_log.predict(test_x_vectors[1])

# step 6: Evaluation

In [ ]:
# Mean Accuracy
# For Support Vector Machine
print(clf_svm.score(test_x_vectors,test_propaganda))
# For Decision Tree
print(clf_dec.score(test_x_vectors,test_propaganda))
# For Decision Naive Bayes
print(clf_gnb.score(test_x_vectors.toarray(),test_propaganda))
# For Logistic Regression
print(clf_log.score(test_x_vectors,test_propaganda))

In [ ]:
# F1 Scores
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_propaganda, clf_svm.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

# For Support Decision Tree
print(f1_score(test_propaganda,clf_dec.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Support Naive Bayes
print(f1_score(test_propaganda,clf_gnb.predict(test_x_vectors.toarray()),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Logistic Regression
print(f1_score(test_propaganda,clf_log.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


In [ ]:
import numpy as np
# check number of positive and negative classes in training set
print("Number of NEGATIVE in training set:")
print(train_propaganda.count(Propaganda.NEGATIVE))
print("Number of POSITIVE in training set:")
print(train_propaganda.count(Propaganda.POSITIVE))


# check number of positive and negative classes in test set
y = np.array(test_propaganda)
number_of_NEGATIVE = (y == "NEGATIVE").sum()
number_of_POSITIVE = (y == "POSITIVE").sum()
print("\nNumber of NEGATIVE in test set:")
print(number_of_NEGATIVE)
print("Number of POSITIVE in test set:")
print(number_of_POSITIVE)

# check number of positive and negative classes in development set

y = np.array(development_propaganda)
Dnumber_of_NEGATIVE = (y == "NEGATIVE").sum()
Dnumber_of_POSITIVE = (y == "POSITIVE").sum()
print("\nNumber of NEGATIVE in development set:")
print(Dnumber_of_NEGATIVE)
print("Number of POSITIVE in development set:")
print(Dnumber_of_POSITIVE)


In [ ]:
test_set =['this is advertisement','bad book do not buy','horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

# step 7: Tuning our model (with Grid Search)

In [ ]:
from sklearn.model_selection import GridSearchCV

# tuned = svm.SVC()
parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32,64)}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv=10)
clf.fit(train_x_vectors,train_propaganda)
clf.cv_results_


In [ ]:
import pandas as pd

df = pd.DataFrame(clf.cv_results_)
df

In [ ]:
df[['param_C','param_kernel','mean_test_score']]

In [ ]:
clf.best_params_